In [23]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = '../model/bottleneck_fc_model.h5'
train_data_dir = '../data/Pandora18K_small_train_val_split/train'
validation_data_dir = '../data/Pandora18K_small_train_val_split/val'
nb_train_samples = 1447
nb_validation_samples = 353
epochs = 50
batch_size = 16


In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)

# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)
bottleneck_features_train = model.predict_generator(
    generator, (nb_train_samples // batch_size)+1)
np.save(open('../model/bottleneck_features_train.npy', 'w'),
        bottleneck_features_train)

generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)
bottleneck_features_validation = model.predict_generator(
    generator, (nb_validation_samples // batch_size)+1)
np.save(open('../model/bottleneck_features_validation.npy', 'w'),
        bottleneck_features_validation)

In [ ]:
from keras.utils.np_utils import to_categorical

train_data = np.load(open('../model/bottleneck_features_train.npy'))
#full
# train_labels = np.array([0]*688+[1]*563+[2]*636+[3]*662+[4]*781+[5]*666+[6]*716+[7]*937+[8]*1024+[9]*1035+[10]*777+[11]*845+[12]*566+[13]*970+[14]*854+[15]*829+[16]*831+[17]*882)
#small
train_labels = np.array([0]*83+[1]*83+[2]*79+[3]*85+[4]*85+[5]*82+[6]*83+[7]*73+[8]*87+[9]*75+[10]*84+[11]*79+[12]*79+[13]*77+[14]*81+[15]*71+[16]*74+[17]*87)
train_labels = to_categorical(train_labels, num_classes=18)

validation_data = np.load(open('../model/bottleneck_features_validation.npy'))
#full
#validation_labels = np.array([0]*159+[1]*168+[2]*166+[3]*170+[4]*209+[5]*166+[6]*179+[7]*254+[8]*233+[9]*227+[10]*238+[11]*193+[12]*145+[13]*242+[14]*217+[15]*203+[16]*218+[17]*227)
#small
validation_labels = np.array([0]*17+[1]*17+[2]*21+[3]*15+[4]*15+[5]*18+[6]*17+[7]*27+[8]*13+[9]*25+[10]*16+[11]*21+[12]*21+[13]*23+[14]*19+[15]*29+[16]*26+[17]*13)
validation_labels = to_categorical(validation_labels, num_classes=18)

model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(18, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_data, train_labels,
          epochs=epochs,
          batch_size=batch_size,
          validation_data=(validation_data, validation_labels))
model.save_weights(top_model_weights_path,overwrite=True)